<a href="https://colab.research.google.com/github/21jayoon/songpa-study/blob/main/WhichCharacterIam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai

In [4]:
import os
from getpass import getpass

os.environ['OPENAI_API_KEY']=getpass()

··········


In [ ]:
!pip install gradio==4.44.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 16.0 MB/s eta 0:00:00
  Attempting uninstall: gradio-client
    Found existing installation: gradio_client 0.7.3
    Uninstalling gradio_client-0.7.3:
      Successfully uninstalled gradio_client-0.7.3
  Attempting uninstall: gradio
    Found existing installation: gradio 4.10.0
    Uninstalling gradio-4.10.0:
      Successfully uninstalled gradio-4.10.0


In [ ]:
import gradio as gr
from openai import OpenAI
from google.colab import userdata

client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

# 캐릭터 추측 챗봇
def which_character_am_I(message, chat_history):
  if message == "":
    return "", chat_history
  else:
    completion=client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system",
             "content": """
             You are the expert of Arthur Conan Doyle's novel. Especially you are the Sherlockian.
             You have read all of the Sherlock Holme's novel series multiple times
             and know all the characters from Doctor Watson to the minor character that shows once in the novel.
             If the user send some characteristic feature, analyze it and
             derive the most similar character in Sherlock Holmes series, Arthur Conan Doyle's novel.

             The example of input and the response must be Korean,
             and the response will be like this.

             '당신이 보내준 성격을 분석해보니 (유쾌하고 당차지만 엉뚱한 걸 잘 믿어버리는) 성격을 갖고 있군요!
             당신은 셜록 홈즈 시리즈에서 (자베즈 윌슨 Jabez Wilson)으로 활약했을 것 같습니다!
             그는 1892년 출간된 (셜록홈즈의 모험)에 수록된 작품 중 (붉은 머리 연맹)이라는 단편에 사건의 의뢰인으로 나왔습니다!'

             The example should include the summary of input strings, and the similar character with input chracteristic,
             and the story that character appears.

             Also, if the user asking unrelated question with Sherlock Holmes character, respond with

             '죄송합니다. 해당 내용은 현재 챗봇에서 구현할 수 없는 내용입니다'.
             """},
             {"role": "user", "content": message}
        ]
    )
    chat_history.append({"role": "user", "content": message})
    chat_history.append({"role":"assistant", "content": completion.choices[0].message.content})
    return "", chat_history

# 챗 내역 되돌리기
def chat_bot_undo(chat_history):
  if len(chat_history) > 1:
    chat_history.pop()
  return chat_history

# 챗 완전 초기화
def reset_the_chatbot(chat_history):
  chat_history=[[None, "셜로키언인 당신! 셜록 홈즈 세계관에 들어간다면 어떤 캐릭터일까요?"]]
  return chat_history

# 챗봇 레이아웃
with gr.Blocks(theme='earneleh/paris') as app:
  with gr.Tab("나는 셜록홈즈 세계관에서 어떤 캐릭터일까?"):
    gr.Markdown(
        value="""
        # <center> 내가 셜록홈즈 세계관에 들어간다면?</center>
        <center>셜로키언인 당신! 홈즈가 사는 런던에 떨어지게 된다면 어떤 인물로 살게되었을까요?</center>
        """
    )

    sh_chatbot=gr.Chatbot(
        value=[{"role": "assistant", "content": "셜로키언인 당신! 셜록 홈즈 세계관에 들어간다면 어떤 캐릭터일까요?"}],
        show_label=False,
        type="messages"
    )
    with gr.Row():
      user_input=gr.Text(
          lines=1,
          placeholder="성격을 입력하세요.",
          container=False,
          scale=9
      )
      send_btn=gr.Button(
          value="보내기",
          scale=1,
          variant="primary",
          icon="https://cdn-icons-png.flaticon.com/128/12439/12439334.png"
      )
    with gr.Row():
      gr.Button(value="되돌리기").click(fn=chat_bot_undo, inputs=sh_chatbot, outputs=sh_chatbot)
      gr.Button(value="초기화").click(fn=reset_the_chatbot, inputs=sh_chatbot, outputs=sh_chatbot)

    # 입력 값 보내는 방법1
    send_btn.click(fn=which_character_am_I, inputs=[user_input, sh_chatbot], outputs=[user_input, sh_chatbot])
    # 입력 값 보내는 방법2
    user_input.submit(fn=which_character_am_I, inputs=[user_input, sh_chatbot], outputs=[user_input, sh_chatbot])


app.launch(share=True, debug=True)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


theme_schema%400.0.1.json: 0.00B [00:00, ?B/s]

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://c622497d3eec3e48fc.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
